In [ ]:
import pandas as pd 
import os
import json
import numpy as np
import re 
from sklearn.base import TransformerMixin , BaseEstimator

In [ ]:
credits = pd.read_csv(r'C:\Users\saadb\recommendation_project\recommendation_model\dataset\credits.csv')

In [ ]:
kewords = pd.read_csv(r'C:\Users\saadb\recommendation_project\recommendation_model\dataset\keywords.csv')

In [ ]:
credits.head(5)

In [ ]:
kewords.head(5)

In [ ]:
credits.info()

In [ ]:
credits.isna().sum()

In [ ]:
credits.cast[0]

In [ ]:
def handle_json(x):
    try :
        
        x = x.replace("'",'"')
    
        x = json.loads(x)
        
    except :
        return np.nan
    
    return x

In [ ]:
number_of_nones = re.findall(r'\bNone\b',credits['cast'][1])
for i in range(len(number_of_nones)):
    match = re.search(r'\bNone\b',credits['cast'][1])
    if match:
        credits['cast'][1] = credits['cast'][1].replace('None','"missing"')

In [ ]:
def handle_json_better(x):
    number_of_nones = re.findall(r'\bNone\b',x)
    
    for i in range(len(number_of_nones)):
        match = re.search(r'\bNone\b',x)
        if match :
            x = x.replace('None' , '"missing"')
    
    try :
        
        x = x.replace("'",'"')
    
        x = json.loads(x)
        
    except :
        
        x = np.nan
        
    return x
    

In [ ]:
handle_json_better(credits['cast'][0])

In [ ]:
credits['cast'] = credits['cast'].apply(lambda x : handle_json_better(x))

In [ ]:
credits['cast']

In [ ]:
def get_cast_names(x):
    y = []
    count = 0
    if x == []:
        return np.nan
    elif x == '':
        return np.nan
    else:
        for i in x:
            if count < 4:
                y.append(i['name'])
                count += 1
        x = '|'.join(y)
        return x
    
        if isinstance(x,list):
            return get_cast_names(x)
    

In [ ]:
def get_main_cast_names(x):
    if x == []:
        return np.nan
    elif x == '':
        return np.nan
    else :
        if x[0]['name'] == '' :
            return np.nan
        elif x[0]['name']:
            return x[0]['name']
    return x

In [ ]:
def get_main_cast_gender(x):
    if x == []:
        return np.nan
    elif x == '':
        return np.nan
    else :
        if x[0]['gender'] == 1:
            return 'F'
        elif x[0]['gender'] == 2:
            return 'M'
        else :
            return np.nan
        
    return x

In [ ]:
credits['cast_names'] = credits['cast'].apply(lambda x : get_cast_names(x) if isinstance(x,list) else np.nan)

In [ ]:
credits['cast_main_name'] = credits['cast'].apply(lambda x : get_main_cast_names(x) if isinstance(x,list) else np.nan)

In [ ]:
credits['cast_main_gender'] = credits['cast'].apply(lambda x : get_main_cast_gender(x) if isinstance(x,list) else np.nan)

In [ ]:
credits['crew'] = credits['crew'].apply(lambda x : handle_json_better(x))

In [ ]:
def get_director(x):
    if x == []:
        return np.nan
    elif x == '':
        return np.nan
    else :
        for i in x:
            if i['department'] == 'Directing':
                x = i['name']
        return x

In [ ]:
def get_writer(x):
    y = []
    if x == []:
        return np.nan
    elif x == '':
        return np.nan
    else :
        for i in x:
            if i['department'] == 'Writing':
                y.append(i['name'])
        x = '|'.join(y)
        return x

In [ ]:
def get_producer(x):
    if x == []:
        return np.nan
    elif x == '':
        return np.nan
    else :
        for i in x:
            if i['department'] == 'Production':
                x = i['name']
        return x 

In [ ]:
credits['Director'] = credits['crew'].apply(lambda x : get_director(x) if isinstance(x,list) else np.nan)

In [ ]:
credits['Writers'] = credits['crew'].apply(lambda x : get_writer(x) if isinstance(x,list) else np.nan)

In [ ]:
credits['Producer'] = credits['crew'].apply(lambda x : get_producer(x) if isinstance(x,list) else np.nan)

In [ ]:
credits['Producer'] = credits['Producer'].apply(lambda x : x if isinstance(x,str) else np.nan)

In [ ]:
credits.info()

now we finished creating something useful from the json format we got we extracted main charater name and first 4 cast from the whole actors from cast and from crew we extracted producer , writer and director also main cast gender from the cast column .

In [ ]:
def get_keywords(x):
    y = []
    if x == []:
        return np.nan
    elif x == '':
        return np.nan
    else :
        for i in x:
            y.append(i['name'])
        x = '|'.join(y)
        return x

In [ ]:
kewords['keywords'] = kewords['keywords'].apply(lambda x : handle_json_better(x))

In [ ]:
kewords['keywords'] = kewords['keywords'].apply(lambda x : get_keywords(x) if isinstance(x,list) else np.nan)

In [ ]:
movies_metadata = pd.read_csv(r'C:\Users\saadb\recommendation_project\jupyter_notebook\datasets\movies_cleaned.csv')

In [ ]:
movies_metadata.head(3)

In [ ]:
movies_metadata.info()

In [ ]:
movies_metadata.dropna(subset=['id'],axis='rows',inplace=True)

In [108]:
movies_metadata['id'] = movies_metadata['id'].astype(int,errors='raise')

In [ ]:
y = []
for i in movies_metadata['id']:
    if '-' in i:
        y.append(i)  
y        

In [ ]:
movies_metadata.drop(19719,inplace=True)

In [ ]:
movies_metadata.drop(29474,inplace=True)

In [ ]:
movies_metadata.drop(35552,inplace=True)

In [107]:
movies_metadata['id'].dtype

dtype('O')

In [ ]:
movies_metadata.id.value_counts()

In [109]:
movies_metadata[movies_metadata.duplicated(subset=['id'],keep=False)].sort_values('id')

,adult,belongs_to_collection,budget,genres,id,original_language,original_title,overview,popularity,production_companies,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count


In [ ]:
movies_metadata.drop_duplicates(subset=['id'],inplace=True)

In [110]:
movies_metadata[~movies_metadata.id.isin(credits.id)]

,adult,belongs_to_collection,budget,genres,id,original_language,original_title,overview,popularity,production_companies,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
42828,False,NaN,NaN,missing,401840,es,School's out,Two high school kids mentored by a nightclub o...,0.207775,missing,...,2017-05-30,NaN,NaN,missing,Released,NaN,School's out,False,0.0,0.0


In [ ]:
#droping row that we dont have enogh information on and we dont have it on crew and cast 


In [111]:
movies_metadata.drop(42828,inplace=True)

In [112]:
credits[~credits.id.isin(movies_metadata.id)]

,cast,crew,id,cast_names,cast_main_name,cast_main_gender,Director,Writers,Producer
5636,"[{'cast_id': 1, 'character': 'Nick Monroe', 'c...","[{'credit_id': '52fe45c5c3a368484e06e10f', 'de...",29072,Jamie Gillis|Samantha Grey|Ryan Hilliard|Richa...,Jamie Gillis,M,Joel M. Reed,Joel M. Reed,Lorin E. Price
13032,"[{'cast_id': 2, 'character': 'Tetsuo', 'credit...","[{'credit_id': '52fe4e639251416c75154d21', 'de...",225614,Masatoshi Nagase|Rentarô Mikuni|Emi Wakui|Miek...,Masatoshi Nagase,M,Yôji Yamada,,NaN
13839,"[{'cast_id': 1001, 'character': 'Himself', 'cr...","[{'credit_id': '52fe48c79251416c750b3149', 'de...",91205,Kanye West|Tego Calderón|Raekwon|Paul Wall,Kanye West,M,Raquel Cepeda,,NaN
14890,[],"[{'credit_id': '52fe4d50c3a36847f825a623', 'de...",174748,NaN,NaN,NaN,Leif Erlsboe,,NaN
16836,[],[],442417,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
43972,[],[],400796,NaN,NaN,NaN,NaN,NaN,NaN
44871,"[{'cast_id': 0, 'character': '', 'credit_id': ...",[],334795,Viktor Tsoy,Viktor Tsoy,NaN,NaN,NaN,NaN
45080,[],[],439314,NaN,NaN,NaN,NaN,NaN,NaN
45348,[],[],335251,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
movies_metadata = movies_metadata.merge(credits , how = 'left' , left_on = 'id' , right_on = 'id')

In [114]:
movies_metadata[~movies_metadata.id.isin(kewords.id)]

,adult,belongs_to_collection,budget,genres,id,original_language,original_title,overview,popularity,production_companies,...,vote_average,vote_count,cast,crew,cast_names,cast_main_name,cast_main_gender,Director,Writers,Producer


In [115]:
movies_metadata = movies_metadata.merge(kewords , how = 'left' , left_on = 'id' , right_on = 'id')

In [118]:
movies_metadata.head(3)

,adult,belongs_to_collection,budget,genres,id,original_language,original_title,overview,popularity,production_companies,...,video,vote_average,vote_count,cast_names,cast_main_name,cast_main_gender,Director,Writers,Producer,keywords
0,False,Toy Story Collection,30.0,Animation|Comedy|Family,862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,...,False,7.7,5415.0,Tom Hanks|Tim Allen|Don Rickles|Jim Varney,Tom Hanks,M,NaN,NaN,NaN,jealousy|toy|boy|friendship|friends|rivalry|bo...
1,False,NaN,65.0,Adventure|Fantasy|Family,8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,TriStar Pictures|Teitler Film|Interscope Commu...,...,False,6.9,2413.0,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,Robin Williams,M,Joe Johnston,Jonathan Hensleigh|Chris van Allsburg|Greg Tay...,William Teitler,NaN
2,False,Grumpy Old Men Collection,NaN,Romance|Comedy,15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.712900,Warner Bros.|Lancaster Gate,...,False,6.5,92.0,Walter Matthau|Jack Lemmon|Ann-Margret|Sophia ...,Walter Matthau,M,Howard Deutch,Mark Steven Johnson|Mark Steven Johnson,NaN,fishing|best friend|duringcreditsstinger|old men


In [117]:
movies_metadata.drop(['cast','crew'],axis=1,inplace=True)

In [120]:
movies_metadata.to_csv(path_or_buf=r'C:\Users\saadb\recommendation_project\jupyter_notebook\datasets\complete_dataset_.csv',index=False)